In [ ]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from scipy import integrate
import control as ctrl



In [ ]:

def Perf_bound(RNN):
    A,B,C,b,c=rnn.Weights()
    At=np.transpose(A)
    Q=np.transpose(C)@C
    Bt=np.transpose(B)
    P=ctrl.dlyap(A,Q)
    return np.trace(Bt@P@B)

def vdp1(t, x):
    return np.array([-x[1], - .3*x[1] + x[0]])

res=500
seq_len=20
t0, t1 = 0, 20                # start and end
t = np.linspace(t0, t1, res+seq_len)  # the points of evaluation of solution
x0 = [2, 0]                   # initial value
x = np.zeros((len(t), len(x0)))   # arrax for solution
x[0, :] = x0
r = integrate.ode(vdp1).set_integrator("dopri5")  # choice of method
r.set_initial_value(x0, t0)   # initial values
for i in range(1, t.size):
    x[i, :] = r.integrate(t[i])+np.random.normal(0,0.03,(2,)) # get one more value, add it to the arrax
    if not r.successful():
        raise RuntimeError("Could not integrate")
plt.plot(t, x)
plt.show()

def createInputs(x,res,seq_len):
    data_list=[]
#     target_list=[]
    for i in range(res-1):
        data_list.append([x[i:seq_len+i,:],x[seq_len+i+1,:]])
    return data_list


In [ ]:

def Rbstdiff(RNN):
    A,B,C,b,c=rnn.Weights()
    At=np.transpose(A)
    Q=np.transpose(C)@C
    Bt=np.transpose(B)
    P=ctrl.dlyap(A,Q)
    I=np.identity(len(A))
    dB=2*P@B
    dC=-2*C@B@Bt@ np.linalg.inv(I-At@A)
    dA=2*B@Bt@P@A @np.linalg.inv(I-At@A)
    return dA,dB,dC
    
    

# ================================

class RNN:
  # A many-to-one Vanilla Recurrent Neural Network.
    def __init__(self, input_size, output_size, hidden_size=64):
        # Weights
        self.Whh = random.randn(hidden_size, hidden_size) / 1000
        self.Wxh = random.randn(hidden_size, input_size) / 1000
        self.Why = random.randn(output_size, hidden_size) / 1000

        # Biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
    
        h = np.zeros((self.Whh.shape[0], 1))

        self.last_inputs = inputs
        self.last_hs = { 0: h }

        # Perform each step of the RNN
        for i, xin in enumerate(inputs):
            xin=xin[...,np.newaxis]
            h = np.tanh(self.Wxh @ xin + self.Whh @ h + self.bh)
            self.last_hs[i + 1] = h

        # Compute the output
        y = self.Why @ h + self.by

        return y, h

    def backprop(self, d_y,epoch, learn_rate=2e-2):
        n = len(self.last_inputs)

        # Calculate dL/dWhy and dL/dby.
        d_Why = d_y @ self.last_hs[n].T
        d_by = d_y

        # Initialize dL/dWhh, dL/dWxh, and dL/dbh to zero.
        d_Whh = np.zeros(self.Whh.shape)
        d_Wxh = np.zeros(self.Wxh.shape)
        d_bh = np.zeros(self.bh.shape)

        # Calculate dL/dh for the last h.
        # dL/dh = dL/dy * dy/dh
        d_h = self.Why.T @ d_y

        # Backpropagate through time.
        for t in reversed(range(n)):
            # An intermediate value: dL/dh * (1 - h^2)
            temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)
#             print(temp.shape)
            # dL/db = dL/dh * (1 - h^2)
            d_bh += temp

            # dL/dWhh = dL/dh * (1 - h^2) * h_{t-1}
            d_Whh += temp @ self.last_hs[t].T

            # dL/dWxh = dL/dh * (1 - h^2) * x
            last_x=self.last_inputs[t]
            last_x=last_x[...,np.newaxis]
            d_Wxh += temp @ last_x.T

            # Next dL/dh = dL/dh * (1 - h^2) * Whh
            d_h = self.Whh @ temp

        # Clip to prevent exploding gradients.
        for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
            np.clip(d, -1, 1, out=d)

        # Update weights and biases using gradient descent.
        # adding the regulization terms -----------------------------------------
        dA,dB,dC=Rbstdiff(self)
        gamma=0
#         gamma=.1/np.log(epoch/10+2)
        learn_rate2=learn_rate*gamma
        self.Whh -= learn_rate * d_Whh+learn_rate2 *dA
        self.Wxh -= learn_rate * d_Wxh+learn_rate2 *dB
        self.Why -= learn_rate * d_Why+learn_rate2 *dC
        self.bh -= learn_rate * d_bh
        self.by -= learn_rate * d_by
    def Weights(self):
        return self.Whh,self.Wxh,self.Why,self.bh,self.by


# ================================
    
rnn = RNN(2, 2, hidden_size=8)


def processData(x, backprop=True,batch_size=500,res=500,seq_len=20):
    X_train=createInputs(x,res,seq_len)
    loss = 0
    d_L_d_y=np.zeros([2,1])

    random.shuffle(X_train)
    for xd in X_train[0:batch_size]:
        inputs = xd[0]
        target = xd[1].reshape(-1,1)

        out, _ = rnn.forward(inputs)
        diff=(out - target)
        loss += (diff**2).mean(axis=0)

        if backprop:
            d_L_d_y += 2*(diff)
            rnn.backprop(d_L_d_y,epoch+1)
        return loss 



# Training loop
M_epoch=5000
for epoch in range(M_epoch):
    
    train_loss = processData(x,backprop=True)
    
    if epoch % (M_epoch/10) == ((M_epoch/10)- 1):
        print('--- Epoch %d' % (epoch + 1))
        print('Train:\tLoss  ' , (train_loss))
        print('Train:\t Perf Bound',Perf_bound(rnn))
        print('Train:\t L2 norm of A',np.linalg.norm(rnn.Whh,2))



In [ ]:
def Prediction(x,sig,res,seq_len):
    y_pre=np.zeros([res-1,2])
    i=0
    x_n=x+np.random.normal(0,sig,x.shape)
    X=createInputs(x_n,res,seq_len)
    for xd in X:
        inputs = xd[0]
        target = xd[1].reshape(-1,1)
        out, _ = rnn.forward(inputs)
        y_pre[i]=out.reshape(-1,2)
        i+=1
    return y_pre



perf,perfb=0,0
for sigma in np.arange(0,.5,1/20):
    yp=Prediction(x,0,res,seq_len)
    yn=Prediction(x,sigma,res,seq_len)
    perf=np.append(perf,((yp-yn)*(yp-yn)).mean(axis=None))
    perfb=np.append(perfb,Perf_bound(RNN)*sigma**2)

plt.figure(figsize=(10, 4))
plt.plot(perf)
plt.plot(perfb)
A,B,C,b,c=rnn.Weights()
# mng = plt.get_current_fig_manager()
# mng.frame.Maximize(True)
plt.figure(figsize=(12, 6))

yp=Prediction(x,0,res,seq_len)
yn=Prediction(x,sigma,res,seq_len)
plt.plot(x[seq_len:])
plt.plot(yp)
plt.plot(yn)

In [ ]:
T=np.linspace(0,M_epoch,M_epoch)
gam=.1/(np.log(T/10+2))
plt.plot(gam)